In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [7]:
df = pd.read_csv("Watertypes_data.csv")#, encoding='cp1252')
df

,id,Name,Paper,Water type,pH,Conductivity (uS),SO4 (mg/l),Cl (mg/l),K (mg/l),Na (mg/l),Ca (mg/l),Mg (mg/l),Concentration (mg/l),HD_0,HD_60,HD_diff,ksed (1/h)
0,1,Santa Barbara seawater,Keller,seawater,8.05,37400.0,3133.00,19333.00,377.80,10620.00,398.00,1361.40,10.00,NaN,NaN,NaN,0.39800
1,2,artificial seawater,Keller,seawater,7.99,36300.0,2550.00,17667.00,339.80,9726.00,376.00,1225.40,10.00,477.51,1227.25,749.74,0.05960
2,3,artificial seawater,Keller,seawater,7.99,36300.0,2550.00,17667.00,339.80,9726.00,376.00,1225.40,50.00,909.19,1950.62,1041.43,0.20500
3,4,artificial seawater,Keller,seawater,7.99,36300.0,2550.00,17667.00,339.80,9726.00,376.00,1225.40,100.00,886.48,1948.86,1062.38,0.38300
4,5,artificial seawater,Keller,seawater,7.99,36300.0,2550.00,17667.00,339.80,9726.00,376.00,1225.40,200.00,886.48,1948.86,1062.38,0.66900
5,6,Bodeg Bay seawater,Keller,seawater,8.08,39300.0,3167.00,19333.00,365.90,10010.00,378.40,1191.00,10.00,NaN,NaN,NaN,0.00663
6,7,USCB lagoon,Keller,lagoon,8.90,23733.0,1400.00,1011.00,190.40,5031.00,213.10,581.00,10.00,NaN,NaN,NaN,0.20300
7,8,Santa Paula groundwater,Keller,groundwater,7.90,3997.0,1900.00,172.60,14.28,293.70,447.10,224.90,10.00,NaN,NaN,NaN,0.23100
8,9,Santa Clara river water,Keller,river water,8.33,4507.0,260.00,125.10,3.11,50.18,110.70,34.08,10.00,NaN,NaN,NaN,0.27000
9,10,estero effluent,Keller,treated effluent,7.68,2780.0,306.67,366.70,35.04,377.90,102.70,49.81,10.00,NaN,NaN,NaN,0.01550


In [8]:
columns_names = df.columns
columns_names

Index(['id', 'Name', 'Paper', 'Water type', 'pH', 'Conductivity (uS)',
       'SO4 (mg/l)', 'Cl (mg/l)', 'K (mg/l)', 'Na (mg/l)', 'Ca (mg/l)',
       'Mg (mg/l)', 'Concentration (mg/l)', 'HD_0', 'HD_60', 'HD_diff',
       'ksed (1/h)'],
      dtype='object')

In [9]:
df = df[df.columns[~df.columns.isin(['id', 'Name', 'Paper', 'Water type','HD_0', 'HD_60', 'HD_diff'])]]
print("The df dataframe shape is: ", df.shape)
df.head()

The df dataframe shape is:  (44, 10)


,pH,Conductivity (uS),SO4 (mg/l),Cl (mg/l),K (mg/l),Na (mg/l),Ca (mg/l),Mg (mg/l),Concentration (mg/l),ksed (1/h)
0,8.05,37400.0,3133.0,19333.0,377.8,10620.0,398.0,1361.4,10.0,0.3980
1,7.99,36300.0,2550.0,17667.0,339.8,9726.0,376.0,1225.4,10.0,0.0596
2,7.99,36300.0,2550.0,17667.0,339.8,9726.0,376.0,1225.4,50.0,0.2050
3,7.99,36300.0,2550.0,17667.0,339.8,9726.0,376.0,1225.4,100.0,0.3830
4,7.99,36300.0,2550.0,17667.0,339.8,9726.0,376.0,1225.4,200.0,0.6690


In [10]:
x = df[df.columns[~df.columns.isin(['ksed (1/h)'])]]

scaler = MinMaxScaler()

x_scaled = scaler.fit_transform(x.to_numpy())
x_scaled = pd.DataFrame(x_scaled, columns=[
    'pH', 'Conductivity (uS)',
    'SO4 (mg/l)', 'Cl (mg/l)', 'K (mg/l)', 'Na (mg/l)', 'Ca (mg/l)',
    'Mg (mg/l)', 'Concentration (mg/l)'])

x_scaled.head()

,pH,Conductivity (uS),SO4 (mg/l),Cl (mg/l),K (mg/l),Na (mg/l),Ca (mg/l),Mg (mg/l),Concentration (mg/l)
0,0.782051,0.781485,0.970674,0.918316,0.809005,0.911765,0.890157,0.968862,0.049952
1,0.766667,0.758373,0.790042,0.839181,0.727612,0.835004,0.840940,0.872069,0.049952
2,0.766667,0.758373,0.790042,0.839181,0.727612,0.835004,0.840940,0.872069,0.249962
3,0.766667,0.758373,0.790042,0.839181,0.727612,0.835004,0.840940,0.872069,0.499975
4,0.766667,0.758373,0.790042,0.839181,0.727612,0.835004,0.840940,0.872069,1.000000


In [11]:
y = df['ksed (1/h)']
print("The y size is: ", y.size)
y.head()

The y size is:  44


0    0.3980
1    0.0596
2    0.2050
3    0.3830
4    0.6690
Name: ksed (1/h), dtype: float64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.1, random_state=0)
print(f'The shape of x_train is: {x_train.shape[0]}x{x_train.shape[1]}')
print(f'The shape of x_test is: {x_test.shape[0]}x{x_test.shape[1]}')

The shape of x_train is: 39x9
The shape of x_test is: 5x9


In [13]:
RF = RandomForestRegressor(n_estimators = 10,
                           criterion = 'squared_error',
                           random_state=0, 
                           )
RF
RF.fit(x_train, y_train)
#print(r2_score(y_test, RF.predict(x_test)))

RandomForestRegressor(n_estimators=10, random_state=0)

In [14]:
y_train_pred = RF.predict(x_train)
print(f'Train MSE = {mean_squared_error(y_train, y_train_pred)}' )
print(f'Train R^2 = {RF.score(x_train, y_train)}' ) 
y_test_pred = RF.predict(x_test)
print(f'Test MSE = {mean_squared_error(y_test, y_test_pred)}' )
print(f'Test R^2 = {RF.score(x_test, y_test)}' ) 

Train MSE = 0.0016695534580769228
Train R^2 = 0.8337199376823278
Test MSE = 0.03078369266239999
Test R^2 = 0.5279043249303066


In [15]:
cv5_scores = cross_val_score(RF,x_train,y_train, cv=5, scoring = 'r2')
print(f'5-fold cross validation mean R^2 = {np.mean(cv5_scores)}')

5-fold cross validation mean R^2 = -1.9484862107330585


In [16]:
def evaluate(best_candidate, new_candidate):
    print('Overall Best Grid Search Candidate')
    print(f'Train R^2 = {r2_score(y_train, best_candidate.predict(x_train))}')
    print(f'Test R^2 = {r2_score(y_test, best_candidate.predict(x_test))}')
    print('New Grid Search Best Candidate')
    print(f'Train R^2 = {r2_score(y_train, new_candidate.predict(x_train))}')
    print(f'Test R^2 = {r2_score(y_test, new_candidate.predict(x_test))}')

In [17]:
# Tuning of Random Forest parameters with Grid search
param_grid = {'bootstrap': [True],
              'max_depth': [10],
              'min_samples_leaf': [1],
              'min_samples_split': [2],
              'n_estimators': [500]}

RF_grid = RandomForestRegressor(random_state=0)
grid_search = GridSearchCV(estimator = RF_grid, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)   

grid_search.fit(x_train, y_train)
           
grid_search.best_params_#, grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


{'bootstrap': True,
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

In [19]:
best_candidate = grid_search.best_estimator_
new_candidate = grid_search.best_estimator_

evaluate(best_candidate, new_candidate)

if r2_score(y_test, new_candidate.predict(x_test)) > r2_score(y_test, best_candidate.predict(x_test))  :
    best_candidate = new_candidate
    best_params = grid_search.best_params_
    print("New best candidate detected")

Overall Best Grid Search Candidate
Train R^2 = 0.889229557959575
Test R^2 = 0.5727372687766961
New Grid Search Best Candidate
Train R^2 = 0.889229557959575
Test R^2 = 0.5727372687766961


In [ ]:
best_params

In [ ]:
importances = best_candidate.feature_importances_
predictors_names = x_train.columns
forest_importances = pd.Series(importances, index=predictors_names)
forest_importances[forest_importances>0].sort_values(ascending=False)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
fig, ax = plt.subplots()
forest_importances[0:15].sort_values(ascending=False).plot.bar(ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()